In [ ]:
from datasets import load_dataset
from datasets import concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
from transformers import DataCollatorForSeq2Seq
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

import os
metric = evaluate.load("rouge")

In [ ]:
from dotenv import load_dotenv
load_dotenv()
hugging_face_username = os.environ.get('HUGGING_FACE_USERNAME')

In [ ]:
os.makedirs(f"models/{hugging_face_username}", exist_ok=True)

In [ ]:
class FineTuneLLM1:

    def __init__(self, model_id, dataset_id):

        self.model_id = model_id
        self.dataset_id = dataset_id
        self.label_pad_token_id = -100

    def tokenize_prep(self):
        dataset_id = self.dataset_id
        dataset = load_dataset(dataset_id)
        model_id = self.model_id
        # Load tokenizer of FLAN-t5-base
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        # Load model from the hub
        model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
        # The maximum total input sequence length after tokenization.
        # Sequences longer than this will be truncated, sequences shorter will be padded.
        tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
        max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
        print(f"Max source length: {max_source_length}")
        # The maximum total sequence length for target text after tokenization.
        # Sequences longer than this will be truncated, sequences shorter will be padded."
        tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
        max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
        print(f"Max target length: {max_target_length}")
        label_pad_token_id = self.label_pad_token_id
        return tokenizer, model, dataset, max_source_length, max_target_length, label_pad_token_id

    def preprocess_function(self, sample,padding="max_length"):
        # add prefix to the input for t5
        inputs = ["summarize: " + item for item in sample["dialogue"]]
        # tokenize inputs
        model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
        # Tokenize targets with the `text_target` keyword argument
        labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)
        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length":
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    def postprocess_text(self, preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]
        # rougeLSum expects newline after each sentence
        preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(sent_tokenize(label)) for label in labels]
        return preds, labels

    def compute_metrics(self, eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        # Some simple post-processing
        decoded_preds, decoded_labels = self.postprocess_text(decoded_preds, decoded_labels)
        metric = evaluate.load("rouge")
        result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        result = {k: round(v * 100, 4) for k, v in result.items()}
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        return result


In [ ]:
## Run Training

model_id = "google/flan-t5-small"
dataset_id = "samsum"
version_number = "1" # Change model version if necessary

ftl1 = FineTuneLLM1(model_id, dataset_id)

tokenizer, model, dataset, max_source_length, max_target_length, label_pad_token_id = ftl1.tokenize_prep()

preprocess_function = ftl1.preprocess_function
compute_metrics = ftl1.compute_metrics

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

metric = evaluate.load("rogue")

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

# Hugging Face repository id
model_name = f"{model_id.split('/')[1]}-{dataset_id}--{version_number}"
repository_id = os.path.join('models', hugging_face_username, model_name)

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=True
)
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.save_model(repository_id)

In [ ]:
tokenizer.save_pretrained(repository_id)

In [ ]:
repository_id

In [ ]:
remote_repo_id = os.path.join(*repository_id.split('/')[1:])
remote_repo_id

In [ ]:
model.push_to_hub(remote_repo_id)

In [ ]:
tokenizer.push_to_hub(remote_repo_id)